In [1]:
from tabula import read_pdf
import pandas as pd
import re
import os
# os.environ["JAVA_HOME"] = "/Library/Internet Plug-Ins/JavaAppletPlugin.plugin/Contents/Home"

In [2]:
pdf_dir = 'pdf'
year_dirs = []
for d in os.listdir(pdf_dir):
    if d[0] != '.':
        year_dirs.append(d)

In [3]:
def scrape_stage(stage_data):
    stage_data = stage_data.copy()
    
    # Identify shooter info columns
    shooter_info = [col for col in stage_data.columns if 'Unnamed' not in col]
    
    # Extract shooter details
    shooter_name = re.sub(r'\d+', '', shooter_info[0]).strip().split('DIV:')[0]
    division = next((col.split('DIV:')[1] for col in stage_data.columns if 'DIV:' in col), '')
    class_ = next((col.replace('CLASS:', '').strip() for col in stage_data.columns if 'CLASS:' in col), '')
    factor = next((col.replace('FACTOR:', '').strip() for col in stage_data.columns if 'FACTOR:' in col), '')
    category = next((col.replace('CATEGORY:', '').strip() for col in stage_data.columns if 'CATEGORY:' in col), '')

    # Set new column names based on first row
    stage_data.columns = stage_data.iloc[0]
    stage_data = stage_data.iloc[1:]

    # Identify and split merged columns
    for col in stage_data.columns:
        if "FACTOR PTS A C D" in col:
            split_cols = stage_data[col].str.split(' ', n=4, expand=True)
            split_cols.columns = ["FACTOR", "PTS", "A", "C", "D"]
            stage_data = stage_data.drop(columns=[col]).join(split_cols)
        elif "FACTOR PTS A C" in col:
            split_cols = stage_data[col].str.split(' ', n=3, expand=True)
            split_cols.columns = ["FACTOR", "PTS", "A", "C"]
            stage_data = stage_data.drop(columns=[col]).join(split_cols)
        elif "FACTOR PTS A" in col:
            split_cols = stage_data[col].str.split(' ', n=2, expand=True)
            split_cols.columns = ["FACTOR", "PTS", "A"]
            stage_data = stage_data.drop(columns=[col]).join(split_cols)
        elif "FACTOR PTS" in col and "PTS" not in stage_data.columns:
            split_cols = stage_data[col].str.split(' ', n=1, expand=True)
            split_cols.columns = ["FACTOR", "PTS"]
            stage_data = stage_data.drop(columns=[col]).join(split_cols)

    # Add shooter details
    stage_data['SHOOTER'] = shooter_name
    stage_data['DIV'] = division
    stage_data['CLASS'] = class_
    stage_data['POWER_FACTOR'] = factor
    stage_data['CAT'] = category

    # Handle 'Ded. MI' and 'NS PE' columns
    if 'Ded. MI' in stage_data.columns:
        stage_data[['DED', 'MI']] = stage_data['Ded. MI'].str.split(' ', n=1, expand=True)
        stage_data.drop(columns=['Ded. MI'], inplace=True)
    
    if 'NS PE' in stage_data.columns:
        stage_data[['NS', 'PE']] = stage_data['NS PE'].str.split(' ', n=1, expand=True)
        stage_data.drop(columns=['NS PE'], inplace=True)

    # Handle 'OT Time' column
    if 'OT Time' in stage_data.columns:
        if stage_data['OT Time'].str.contains(' ').any():
            stage_data[['OT', 'TIME']] = stage_data['OT Time'].str.split(' ', n=1, expand=True)
        else:
            stage_data['OT'] = ''
            stage_data['TIME'] = stage_data['OT Time']
        stage_data.drop(columns=['OT Time'], inplace=True)

    # Reorder columns
    columns_order = ['SHOOTER', 'DIV', 'CLASS', 'POWER_FACTOR', 'CAT', 'STG', 'FACTOR', 'PTS', 'A', 'C', 'D', 'DED', 'MI', 'NS', 'PE', 'OT', 'TIME']
    stage_data = stage_data[[col for col in columns_order if col in stage_data.columns]]
    stage_data['DIV'] = stage_data['DIV'].str.strip()
    return stage_data

In [4]:
def get_match_data(match_name, match_date, match_level, match_file):
    df_list = read_pdf(match_file, pages="all", multiple_tables=True, java_options="-Djava.awt.headless=true")
    dfs_ = []
    for df_ in df_list:
        dfs_.append(scrape_stage(df_))
    match = pd.concat(dfs_).reset_index(drop=True)
    match['match_name'] = match_name
    match['match_date'] = match_date
    match['match_level'] = match_level
    match.columns = match.columns.str.lower()
    
    match['factor'] = match['factor'].apply(lambda x: x.replace(',', '.')).astype(float)
    match['time'] = match['time'].apply(lambda x: x.replace(',', '.')).astype(float)
    match['pts'] = match['pts'].astype(int)
    match['hit_factor'] = round(match['pts'] / match['time'], 4)
    match['hf_check'] = match['hit_factor'] - match['factor']

    match = match[['match_name', 'match_level', 'match_date', 'shooter', 'div', 'class', 'power_factor', 'cat', 'stg', 'factor', 
                #    'hf_check',
                #    'hit_factor', 
                   'pts', 'a', 'c', 'd', 'ded', 'mi', 'ns', 'pe', 'ot', 'time']]

    # match['div_pts_perc'] = match.groupby(['match_name', 'div', 'stg'])['pts'].transform(lambda x: (x / x.max()))
    # match['div_first_time'] = match.groupby(['match_name', 'div', 'stg'])['time'].transform(lambda x: x[match['hit_factor'] > 0].min() if any(match['hit_factor'] > 0) else None)
    # match['div_time_perc'] = match['div_first_time'] / match['time']
    # match['div_factor_perc'] = match.groupby(['match_name', 'div', 'stg'])['hit_factor'].transform(lambda x: (x / x.max()))
    # match['div_factor_standing'] = match.groupby(['match_name', 'div', 'stg'])['hit_factor'].transform(lambda x: x.rank(method='first', ascending=False))
    
    # match['cls_pts_perc'] = match.groupby(['match_name', 'div', 'class', 'stg'])['pts'].transform(lambda x: (x / x.max()))
    # match['cls_first_time'] = match.groupby(['match_name', 'div', 'class', 'stg'])['time'].transform(lambda x: x[match['hit_factor'] > 0].min() if any(match['hit_factor'] > 0) else None)
    # match['cls_time_perc'] = match['cls_first_time'] / match['time']
    # match['cls_factor_perc'] = match.groupby(['match_name', 'div', 'class', 'stg'])['hit_factor'].transform(lambda x: (x / x.max()))
    # match['cls_factor_standing'] = match.groupby(['match_name', 'div', 'class', 'stg'])['hit_factor'].transform(lambda x: x.rank(method='first', ascending=False))
    
    # match_abcd = match[match['class'].isin(['A', 'B', 'C', 'D'])].copy()
    # match['div_pts_perc_abcd'] = match_abcd.groupby(['match_name', 'div', 'stg'])['pts'].transform(lambda x: (x / x.max()))
    # match['div_first_time_abcd'] = match_abcd.groupby(['match_name', 'div', 'stg'])['time'].transform(lambda x: x[match_abcd['hit_factor'] > 0].min() if any(match_abcd['hit_factor'] > 0) else None)
    # match['div_time_perc_abcd'] = match_abcd['div_first_time'] / match_abcd['time']
    # match['div_factor_perc_abcd'] = match_abcd.groupby(['match_name', 'div', 'stg'])['hit_factor'].transform(lambda x: (x / x.max()))
    # match['div_factor_standing_abcd'] = match_abcd.groupby(['match_name', 'div', 'stg'])['hit_factor'].transform(lambda x: x.rank(method='first', ascending=False))
    
    # match['cls_pts_perc_abcd'] = match_abcd.groupby(['match_name', 'div', 'class', 'stg'])['pts'].transform(lambda x: (x / x.max()))
    # match['cls_first_time_abcd'] = match_abcd.groupby(['match_name', 'div', 'class', 'stg'])['time'].transform(lambda x: x[match_abcd['hit_factor'] > 0].min() if any(match_abcd['hit_factor'] > 0) else None)
    # match['cls_time_perc_abcd'] = match_abcd['cls_first_time'] / match_abcd['time']
    # match['cls_factor_perc_abcd'] = match_abcd.groupby(['match_name', 'div', 'class', 'stg'])['hit_factor'].transform(lambda x: (x / x.max()))
    # match['cls_factor_standing_abcd'] = match_abcd.groupby(['match_name', 'div', 'class', 'stg'])['hit_factor'].transform(lambda x: x.rank(method='first', ascending=False))

    return match

In [5]:
matches = []
for year in year_dirs:
    for pdf in os.listdir(pdf_dir+'/'+year):
        if pdf != '.' and '_verify.pdf' in pdf:
            path = pdf_dir+'/'+year+'/'+pdf
            match_date = pdf[:10]
            match_name = pdf.replace('_verify.pdf', '')
            match_name = match_name.upper()
            match_name = year+match_name[13:]
            match_level = pdf[12:13]
            matches.append(get_match_data(match_name=match_name, match_date=match_date, match_level=match_level, match_file=path))
df = pd.concat(matches).sort_values('match_date')

In [6]:
df.isnull().sum()

match_name      0
match_level     0
match_date      0
shooter         0
div             0
class           0
power_factor    0
cat             0
stg             0
factor          0
pts             0
a               0
c               0
d               0
ded             0
mi              0
ns              0
pe              0
ot              0
time            0
dtype: int64

In [7]:
df.to_csv('IPSC.csv', index=False)